In [1]:
# lab 7 
#home task 
#Question #1 (Coonect four game)

#Creating a Connect Four game using Pygame. In this game, two players (AI and Human) take turns dropping colored discs into a grid. The first player to align four discs horizontally, vertically, or diagonally wins. Your job is to implement the Minimax Algorithm for the AI (playing as the red discs) to choose the optimal move.




import numpy as np
import matplotlib.pyplot as plt
import random

ROW, COL = 6, 7
PLAYER, AI = 1, 2
EMPTY = 0

# Create board
board = np.zeros((ROW, COL), dtype=int)

# Draw board
def draw(board):
    plt.imshow(np.flip(board, 0), cmap='coolwarm')
    plt.show()

# Check win
def win(board, p):
    for r in range(ROW):
        for c in range(COL-3):
            if all(board[r, c+i]==p for i in range(4)): return True
    for r in range(ROW-3):
        for c in range(COL):
            if all(board[r+i, c]==p for i in range(4)): return True
    for r in range(ROW-3):
        for c in range(COL-3):
            if all(board[r+i, c+i]==p for i in range(4)): return True
            if all(board[r+3-i, c+i]==p for i in range(4)): return True
    return False

# Drop piece
def drop(board, col, p):
    for r in range(ROW):
        if board[r,col]==0:
            board[r,col]=p
            return

# Simple AI: choose winning or random move
def ai_move(board):
    for c in range(COL):
        temp=board.copy()
        if temp[0,c]==0:
            drop(temp, c, AI)
            if win(temp, AI): return c
    for c in range(COL):
        temp=board.copy()
        if temp[0,c]==0:
            drop(temp, c, PLAYER)
            if win(temp, PLAYER): return c
    return random.choice([c for c in range(COL) if board[0,c]==0])

# Game loop
game_over = False
turn = PLAYER
draw(board)

while not game_over:
    if turn == PLAYER:
        col = int(input("Enter column (0-6): "))
    else:
        col = ai_move(board)
        print(f"AI chooses column {col}")
    if board[0,col]==0:
        drop(board, col, turn)
        draw(board)
        if win(board, turn):
            print("Player wins!" if turn==PLAYER else "AI wins!")
            game_over = True
        turn = AI if turn==PLAYER else PLAYER


ModuleNotFoundError: No module named 'numpy'

In [2]:
#lab 7 home task 
#Question #2 (Chess game)

#Implement Alpha-Beta Pruning to optimize the decision-making of an AI (White pieces) in a Chess game. The goal is to create a Chess game where the AI uses Alpha-Beta Pruning to choose the best possible move. The game will be played between a human player (Black pieces) and the AI (White pieces), with the AI evaluating its moves using Minimax combined with Alpha-Beta Pruning.

#Objective:
#Implement Alpha-Beta Pruning: Use the Alpha-Beta Pruning technique to optimize the AI’s decision- making in a Chess game.
#Create a Chess game using Pygame: Display the Chess board graphically and allow the player to interact with it.
#Implement a simple evaluation function: Evaluate the game board based on material (the value of the pieces).
#Create the AI (White pieces): Implement the AI using Alpha-Beta Pruning to choose the optimal move.
#Human Player (Black pieces): The player will be able to make moves by clicking on the Chess board or typing in moves.
#Instructions:

#Setup the Chess Board:Use Pygame to display an 8x8 Chess board with pieces in their initial positions.

#Minimax and Alpha-Beta Pruning:

#o	Implement the Minimax Algorithm to simulate possible moves.
#o	Optimize it with Alpha-Beta Pruning to speed up decision-making.

#Board Evaluation:

#o	Create an evaluation function that assigns scores to board positions based on material (piece values).
#o	The AI should maximize its score and minimize the player’s score.

#Game Flow:

#o	Alternate turns between the AI (White) and the player (Black).
#o	After each move, the board should update, and the game should check for checkmate, stalemate, or a draw


import pygame
import chess
import chess.engine
import math
import sys

# Initialize Pygame
pygame.init()
WIDTH, HEIGHT = 480, 480
SQUARE_SIZE = WIDTH // 8
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Chess AI (Alpha-Beta Pruning)")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
LIGHT_BROWN = (240, 217, 181)
DARK_BROWN = (181, 136, 99)

# Piece values
piece_values = {
    chess.PAWN: 10,
    chess.KNIGHT: 30,
    chess.BISHOP: 30,
    chess.ROOK: 50,
    chess.QUEEN: 90,
    chess.KING: 900
}

# Load images
pieces_images = {}
pieces = ['P','N','B','R','Q','K','p','n','b','r','q','k']
for piece in pieces:
    img = pygame.image.load(f"images/{piece}.png")  # Add your piece images in an "images" folder
    pieces_images[piece] = pygame.transform.scale(img, (SQUARE_SIZE, SQUARE_SIZE))

# Create chess board
board = chess.Board()

# Evaluation function (material only)
def evaluate_board(board):
    score = 0
    for piece_type in piece_values:
        score += len(board.pieces(piece_type, chess.WHITE)) * piece_values[piece_type]
        score -= len(board.pieces(piece_type, chess.BLACK)) * piece_values[piece_type]
    return score

# Minimax with alpha-beta pruning
def minimax(board, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board), None
    
    best_move = None
    if maximizingPlayer:
        maxEval = -math.inf
        for move in board.legal_moves:
            board.push(move)
            eval, _ = minimax(board, depth-1, alpha, beta, False)
            board.pop()
            if eval > maxEval:
                maxEval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return maxEval, best_move
    else:
        minEval = math.inf
        for move in board.legal_moves:
            board.push(move)
            eval, _ = minimax(board, depth-1, alpha, beta, True)
            board.pop()
            if eval < minEval:
                minEval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return minEval, best_move

# Draw board
def draw_board(screen, board):
    for row in range(8):
        for col in range(8):
            color = LIGHT_BROWN if (row + col) % 2 == 0 else DARK_BROWN
            pygame.draw.rect(screen, color, pygame.Rect(col*SQUARE_SIZE, row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
            piece = board.piece_at(chess.square(col, 7-row))
            if piece:
                screen.blit(pieces_images[piece.symbol()], (col*SQUARE_SIZE, row*SQUARE_SIZE))

# Main game loop
running = True
selected_square = None
player_turn = True  # Black (human)
depth = 3  # Depth for AI

while running:
    draw_board(screen, board)
    pygame.display.flip()

    if board.is_checkmate():
        print("Checkmate!")
        running = False
    elif board.is_stalemate() or board.is_insufficient_material():
        print("Draw!")
        running = False

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        
        # Player moves (Black)
        if player_turn and event.type == pygame.MOUSEBUTTONDOWN:
            x, y = pygame.mouse.get_pos()
            col = x // SQUARE_SIZE
            row = y // SQUARE_SIZE
            square = chess.square(col, 7-row)
            if selected_square is None:
                if board.piece_at(square) and board.piece_at(square).color == chess.BLACK:
                    selected_square = square
            else:
                move = chess.Move(selected_square, square)
                if move in board.legal_moves:
                    board.push(move)
                    player_turn = False
                selected_square = None

    # AI move (White)
    if not player_turn and not board.is_game_over():
        _, move = minimax(board, depth, -math.inf, math.inf, True)
        if move:
            board.push(move)
        player_turn = True



ModuleNotFoundError: No module named 'pygame'